1. The departure time of *flight i* is later than the landing time of *flight j*
2. The departure airport of *flight i* is in one hour distance of *flight j*'s landing airport.

In [1]:
import numpy as np
import pandas as pd

In [2]:
coordinates = np.array([[565.0,575.0],[25.0,185.0],[345.0,750.0],[945.0,685.0],[845.0,655.0],
                        [880.0,660.0],[25.0,230.0],[525.0,1000.0],[580.0,1175.0],[650.0,1130.0],
                        [1605.0,620.0],[1220.0,580.0],[1465.0,200.0],[1530.0,  5.0],[845.0,680.0],
                        [725.0,370.0],[145.0,665.0],[415.0,635.0],[510.0,875.0],[560.0,365.0],
                        [300.0,465.0],[520.0,585.0],[480.0,415.0],[835.0,625.0],[975.0,580.0],
                        [1215.0,245.0],[1320.0,315.0],[1250.0,400.0],[660.0,180.0],[410.0,250.0],
                        [420.0,555.0],[575.0,665.0],[1150.0,1160.0],[700.0,580.0],[685.0,595.0],
                        [685.0,610.0],[770.0,610.0],[795.0,645.0],[720.0,635.0],[760.0,650.0],
                        [475.0,960.0],[95.0,260.0],[875.0,920.0],[700.0,500.0],[555.0,815.0],
                        [830.0,485.0],[1170.0, 65.0],[830.0,610.0],[605.0,625.0],[595.0,360.0],
                        [1340.0,725.0],[1740.0,245.0]])

In [3]:
coordinates.shape

(52, 2)

In [6]:
def get_distmat(coordinates):
    num = coordinates.shape[0]
    distmat = np.zeros((52,52))
    for i in range(num):
        for j in range(i, num):
            distmat[i][j]= distmat[j][i] = np.linalg.norm(coordinates[i]-coordinates[j])
    return distmat        

In [7]:
distmat = get_distmat(coordinates)

In [10]:
#distmat[3][2]

603.5105632878351

In [11]:
numant = 40

In [12]:
numcity = coordinates.shape[0]

In [15]:
alpha = 1

In [16]:
beta = 5

In [17]:
rho = 0.1

In [18]:
Q = 1

In [19]:
iter = 0

In [20]:
itermax = 200 

In [22]:
np.diag([1, 2, 3])

array([[1, 0, 0],
       [0, 2, 0],
       [0, 0, 3]])

In [23]:
etatable = 1.0/(distmat + np.diag([1e10]*numcity))

In [36]:
pheromonetable = np.ones((numcity,numcity))

In [37]:
pathtable = np.zeros((numant,numcity)).astype(int)

In [38]:
lengthaver = np.zeros(itermax)

In [41]:
lengthbest = np.zeros(itermax)

In [42]:
pathbest = np.zeros((itermax, numcity))

In [65]:
while iter < itermax:
    if numant <= numcity:
        pathtable[:,0] = np.random.permutation(range(0,numcity))[:numant]
    else:
        pathtable[:numcity, 0] = np.random.permutation(range(0,numcity))[:]
        pathtable[numcity:, 0] = np.random.permutation(range(0, numcity))[:numant-numcity]
    length = np.zeros(numant)
    for i in range(numant):
        visiting = pathtable[i,0]
        unvisited = set(range(numcity))
        unvisited.remove(visiting)
        
        for j in range(1, numcity):
            listunvisited = list(unvisited)
            probtrans = np.zeros(len(listunvisited))

            for k in range(len(listunvisited)):
                probtrans[k] = np.power(pheromonetable[visiting][listunvisited[k]], alpha) * np.power(etatable[visiting][listunvisited[k]],beta)
            cumsumprobtrans = (probtrans/sum(probtrans)).cumsum()# ?
            cumsumprobtrans -= np.random.rand()
            k = listunvisited[np.where(cumsumprobtrans>0)[0][0]]
            pathtable[i,j] = k
            unvisited.remove(k)

            length[i] += distmat[visiting][k]
            visiting = k

        length[i] += distmat[visiting][pathtable[i,0]]

    lengthaver[iter] = length.mean()
    if iter == 0:
        lengthbest[iter] = length.min()
        pathbest[iter] = pathtable[length.argmin()].copy()
    else:
        if length.min() > lengthbest[iter-1]:
            lengthbest[iter] = lengthbest[iter-1]
            pathbest[iter] = pathbest[iter-1].copy()
        else:
            lengthbest[iter] = length.min()
            pathbest[iter] = pathtable[length.argmin()].copy()
            print("最短路径{}米".format(lengthbest[iter]))
            print("经过城市{}".format(pathbest[iter]))
            
    change_pheromone_table = np.zeros((numcity,numcity))
    for i in range(numant):
        for j in range(numcity-1):
            change_pheromone_table[pathtable[i,j]][pathtable[i,j+1]] += Q/distmat[pathtable[i,j]][pathtable[i,j+1]]
        change_pheromone_table[pathtable[i,j+1]][pathtable[i,0]] += Q/distmat[pathtable[i,j+1]][pathtable[i,0]]
    pheromonetable = (1-rho)*pheromonetable + change_pheromone_table
    
    iter += 1
    if (iter-1)%20 == 0:
        print("当前迭代次数{}".format(iter-1))


最短路径8896.93242728米
经过城市[31. 48.  0. 21. 30. 17. 43. 33. 34. 35. 38. 39. 37. 23. 47.  4. 14.  5.
  3. 24. 45. 15. 28. 19. 49. 22. 29.  6.  1. 41. 16. 20.  2. 18. 44. 40.
  7.  9.  8. 42. 32. 50. 11. 27. 26. 25. 46. 12. 13. 51. 10. 36.]
最短路径8632.59802731米
经过城市[23. 47.  4. 14.  5.  3. 24. 36. 37. 39. 38. 35. 34. 33. 43. 48. 31. 44.
 18. 40.  7.  8.  9. 42. 32. 50. 11. 27. 26. 25. 46. 13. 12. 51. 10. 45.
 15. 49. 19. 22. 21.  0. 30. 17.  2. 16. 20. 41.  6.  1. 29. 28.]
最短路径8543.12674989米
经过城市[ 6.  1. 41. 20. 29. 22. 19. 49. 15. 43. 33. 34. 35. 38. 39. 37. 23. 47.
  4. 14.  5.  3. 24. 45. 36. 48.  0. 21. 31. 44. 18. 40.  7.  9.  8. 42.
 32. 11. 50. 10. 51. 13. 12. 26. 27. 25. 46. 28. 17. 30.  2. 16.]
当前迭代次数20
最短路径8402.18759857米
经过城市[37. 23. 47.  4. 14.  5.  3. 24. 45. 43. 34. 35. 33. 38. 39. 36. 48. 31.
  0. 21. 17. 30. 20. 22. 19. 49. 15. 28. 29.  1.  6. 41. 16.  2. 44. 18.
 40.  7.  8.  9. 42. 32. 50. 11. 27. 26. 25. 46. 13. 12. 51. 10.]
最短路径8076.29718715米
经过城市[34. 33. 35. 39. 37. 36. 38.

In [59]:
for j in range(10):
    i = j
print(j)

9


In [56]:
np.zeros(numant)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.])